# out_fiberID
* 기존변수 + u-g,g-r,r-i,i-z만 추가변수로 사용
* fiber_ID를 제외하고 진행해봄
* 제출결과 : 0.3793087274 
-> fiberID를 제외해서 0.02상승

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss


import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV, GridSearchCV

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# #colab용
# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/Dacon/천체관측/')

In [4]:
# path = "/content/drive/My Drive/Colab Notebooks/Dacon/천체관측/data/" #colab
path = "./data/" #local
train_df = pd.read_csv(path+'train.csv',index_col=0)
test_df = pd.read_csv(path+'test.csv',index_col=0)
sample_submission_df = pd.read_csv(path+'sample_submission.csv',index_col=0)
pd.options.display.max_columns = 30

In [5]:
train_df.shape, test_df.shape

((199991, 22), (10009, 21))

## DATA Setting

In [6]:
def del_outlier(data, min=0, max=60):
    up_idx_t=()
    dw_idx_t=()
    train_light = data.iloc[:,2:]
    for i in range(len(train_light.columns)):
        col = train_light.columns[i]
        up_idx_t+=tuple(data[data[col]>max].index)
        dw_idx_t+=tuple(data[data[col]<min].index)
    del_idx = set(up_idx_t+dw_idx_t)
    
    return data[~data.index.isin(del_idx)]

In [7]:
train_df = del_outlier(train_df, min=-60, max=60)

In [8]:
train_df.shape

(199805, 22)

In [9]:
column_number = {}
number_columns = {}
for i, column in enumerate(sample_submission_df.columns):
    column_number[column] = i
    number_columns[i] = column
    
    
def to_number(x, dic):
    return dic[x]

train_df['type_num'] = train_df['type'].apply(lambda x: to_number(x, column_number))


In [10]:
def add_minus_feature_all(data,test = False):
    '''
    combination으로 모든 변수들의 조합에 대해서 컬럼생성 
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 64개로 증가
    '''
    from itertools import combinations
    n = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for combi in list(combinations(selected,2)):
            name_1st = str.split(combi[0],'_')[1]
            name_2nd = str.split(combi[1],'_')[1]
            data[mag+"_"+name_1st+"-"+name_2nd] = data[combi[0]]-data[combi[1]]
        n=count
    return data

In [11]:
def add_minus_feature(data,test = False):
    '''
    근접변수에 대해서만 컬럼생성 ex) 'fiberMag_u-g', 'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z'
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 38개로 증가 
    '''
    from itertools import combinations
    n = 0
    m = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for i in range(0,len(selected)):        
            if i>0: 
                name_1st = str.split(m,'_')[1]
                name_2nd = str.split(selected[i],'_')[1]
                data[mag+"_"+name_1st+"-"+name_2nd] = data[m]-data[selected[i]]
            m = selected[i]    
        n=count
    return data

In [12]:
train_df = add_minus_feature_all(train_df)
test_df = add_minus_feature_all(test_df,True)

In [13]:
train_df.shape,test_df.shape

((199805, 63), (10009, 61))

In [14]:
train_df.columns

Index(['type', 'fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
       'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
       'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
       'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
       'modelMag_z', 'type_num', 'psfMag_u-g', 'psfMag_u-r', 'psfMag_u-i',
       'psfMag_u-z', 'psfMag_g-r', 'psfMag_g-i', 'psfMag_g-z', 'psfMag_r-i',
       'psfMag_r-z', 'psfMag_i-z', 'fiberMag_u-g', 'fiberMag_u-r',
       'fiberMag_u-i', 'fiberMag_u-z', 'fiberMag_g-r', 'fiberMag_g-i',
       'fiberMag_g-z', 'fiberMag_r-i', 'fiberMag_r-z', 'fiberMag_i-z',
       'petroMag_u-g', 'petroMag_u-r', 'petroMag_u-i', 'petroMag_u-z',
       'petroMag_g-r', 'petroMag_g-i', 'petroMag_g-z', 'petroMag_r-i',
       'petroMag_r-z', 'petroMag_i-z', 'modelMag_u-g', 'modelMag_u-r',
       'modelMag_u-i', 'modelMag_u-z', 'modelMag_g-r', 'modelMag_g-i',
       'modelMag_g-z', 'modelMag_r-i', 'modelMag_r-z

### CV

In [52]:
X = train_df.drop(columns=['type', 'type_num','fiberID'], axis=1)
y = train_df['type_num']
test_X = test_df.drop(columns = ['fiberID'])

In [53]:
folds = 3
Fold = StratifiedKFold(n_splits=folds, shuffle=True,random_state=42)
kfold =KFold(n_splits=folds, shuffle=True,random_state=42)

In [57]:
2^11

9

In [54]:
params = {"max_depth" : (7,8,9,10),
         "learning_rate": (0.01,0.05, 0.1),
         "num_leaves": (127,255,511,1023),
         "min_child_samples": (30,40,50),
         "lambda_l1": (0,3,5,7,9),
         "lambda_l2": (0,3,5,7,9)}

In [55]:
defaut_lgbmc = lgb.LGBMClassifier(n_estimators=2000,random_state=1029,objective ="multiclass",class_weight = 'balanced')

In [56]:
random_search  = RandomizedSearchCV(defaut_lgbmc,param_distributions=params, cv=5, verbose =100 ,n_iter = 10)

In [43]:
#와 3시간.. fold = 3,cv =5, n_iter =2 
for fold, (train_ids, val_ids) in enumerate(Fold.split(X,y)):
    print(fold, "start")
    eval_set = [(X.iloc[train_ids], y.iloc[train_ids]),(X.iloc[val_ids], y.iloc[val_ids])]
    eval_names = ['train','val']
    random_search.fit(X, y, eval_set = eval_set, eval_names = eval_names, early_stopping_rounds = 100,verbose = 100)

0 start
Fitting 2 folds for each of 2 candidates, totalling 4 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] num_leaves=1023, min_child_samples=30, max_depth=7, learning_rate=0.1, lambda_l2=5, lambda_l1=5 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.418105	val's multi_logloss: 0.422927
[200]	train's multi_logloss: 0.361815	val's multi_logloss: 0.366202
[300]	train's multi_logloss: 0.3333	val's multi_logloss: 0.337554
[400]	train's multi_logloss: 0.316688	val's multi_logloss: 0.320975
[500]	train's multi_logloss: 0.305841	val's multi_logloss: 0.310139
[600]	train's multi_logloss: 0.298565	val's multi_logloss: 0.302898
[700]	train's multi_logloss: 0.293548	val's multi_logloss: 0.297803
[800]	train's multi_logloss: 0.289792	val's multi_logloss: 0.293963
[900]	train's multi_logloss: 0.2872	val's multi_logloss: 0.291397
[1000]	train's multi_logloss: 0.285606	val's multi_logloss: 0.289797
[1100]	trai

1 start
Fitting 2 folds for each of 2 candidates, totalling 4 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] num_leaves=511, min_child_samples=50, max_depth=10, learning_rate=0.05, lambda_l2=0, lambda_l1=3 
Training until validation scores don't improve for 100 rounds
[100]	train's multi_logloss: 0.439785	val's multi_logloss: 0.436849
[200]	train's multi_logloss: 0.357114	val's multi_logloss: 0.354975
[300]	train's multi_logloss: 0.317675	val's multi_logloss: 0.316391
[400]	train's multi_logloss: 0.295327	val's multi_logloss: 0.294383
[500]	train's multi_logloss: 0.281595	val's multi_logloss: 0.281288
[600]	train's multi_logloss: 0.27333	val's multi_logloss: 0.273284
[700]	train's multi_logloss: 0.267588	val's multi_logloss: 0.26771
[800]	train's multi_logloss: 0.263427	val's multi_logloss: 0.263661
[900]	train's multi_logloss: 0.260408	val's multi_logloss: 0.260606
[1000]	train's multi_logloss: 0.258156	val's multi_logloss: 0.258372
[1100]	t

[900]	train's multi_logloss: 0.1215	val's multi_logloss: 0.122134
[1000]	train's multi_logloss: 0.113104	val's multi_logloss: 0.11372
[1100]	train's multi_logloss: 0.106327	val's multi_logloss: 0.106891
[1200]	train's multi_logloss: 0.100712	val's multi_logloss: 0.101241
[1300]	train's multi_logloss: 0.0960584	val's multi_logloss: 0.09655
[1400]	train's multi_logloss: 0.0921609	val's multi_logloss: 0.0925854
[1500]	train's multi_logloss: 0.0888516	val's multi_logloss: 0.0892409
[1600]	train's multi_logloss: 0.0858237	val's multi_logloss: 0.0862053
[1700]	train's multi_logloss: 0.0833091	val's multi_logloss: 0.0836819
[1800]	train's multi_logloss: 0.0810297	val's multi_logloss: 0.0814026
[1900]	train's multi_logloss: 0.0791222	val's multi_logloss: 0.0794735
[2000]	train's multi_logloss: 0.07736	val's multi_logloss: 0.0776853
Did not meet early stopping. Best iteration is:
[2000]	train's multi_logloss: 0.07736	val's multi_logloss: 0.0776853
2 start
Fitting 2 folds for each of 2 candidate

[1300]	train's multi_logloss: 0.0745527	val's multi_logloss: 0.0725261
[1400]	train's multi_logloss: 0.0727093	val's multi_logloss: 0.0707404
[1500]	train's multi_logloss: 0.0715164	val's multi_logloss: 0.0696133
[1600]	train's multi_logloss: 0.0707123	val's multi_logloss: 0.0688601
[1700]	train's multi_logloss: 0.0699631	val's multi_logloss: 0.0681532
[1800]	train's multi_logloss: 0.0695999	val's multi_logloss: 0.0678082
Early stopping, best iteration is:
[1759]	train's multi_logloss: 0.0695999	val's multi_logloss: 0.0678082


In [44]:
params2 = random_search.best_params_

In [45]:
params2

{'num_leaves': 1023,
 'min_child_samples': 40,
 'max_depth': 9,
 'learning_rate': 0.1,
 'lambda_l2': 0,
 'lambda_l1': 3}

In [46]:
gmodel = random_search.best_estimator_

In [47]:
gmodel

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', lambda_l1=3,
               lambda_l2=0, learning_rate=0.1, max_depth=9,
               min_child_samples=40, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=1023,
               objective='multiclass', random_state=1029, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [48]:
gmodel.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', lambda_l1=3,
               lambda_l2=0, learning_rate=0.1, max_depth=9,
               min_child_samples=40, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=2000, n_jobs=-1, num_leaves=1023,
               objective='multiclass', random_state=1029, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [49]:
y_pred = gmodel.predict_proba(test_X)

In [50]:
y_pred

array([[2.35583178e-05, 2.06124297e-05, 6.15106766e-05, ...,
        1.31041538e-04, 1.54679612e-04, 2.29873919e-05],
       [7.98886474e-05, 6.71206251e-05, 7.70500723e-06, ...,
        5.25065976e-05, 6.04471483e-05, 8.65629305e-05],
       [1.50800218e-05, 2.41174122e-06, 3.97773548e-07, ...,
        1.95714026e-06, 4.39013894e-06, 3.13118895e-06],
       ...,
       [5.34015004e-05, 1.32591143e-04, 7.00782471e-06, ...,
        1.32663784e-05, 2.01321806e-05, 6.22334042e-02],
       [2.59124873e-06, 2.53303517e-06, 4.72557051e-07, ...,
        4.61696693e-07, 1.80796863e-06, 1.14154966e-04],
       [6.24877089e-05, 9.74950612e-07, 6.08538955e-07, ...,
        1.28538947e-06, 1.47364012e-06, 4.24914262e-06]])

## Modeling

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=[15, 4])

# Plot the log loss during training
axs[0].plot(evaluation_results['Train']['multi_logloss'], label='Train')
axs[0].plot(evaluation_results['Test']['multi_logloss'], label='Test')
axs[0].set_ylabel('Log loss')
axs[0].set_xlabel('Boosting round')
axs[0].set_title('Training performance')
axs[0].legend()

# Plot feature importance
importances = pd.DataFrame({'features': lgmb_model.feature_name(), 
                            'importance': lgmb_model.feature_importance()}).sort_values('importance', ascending=False).head(10)
axs[1].bar(x=np.arange(len(importances)), height=importances['importance'])
axs[1].set_xticks(np.arange(len(importances)))
axs[1].set_xticklabels(importances['features'])
axs[1].set_ylabel('Feature importance (# times used to split)')
axs[1].set_title('Feature importance')
plt.xticks(rotation=45)

plt.show()

In [ ]:
train_y_pred = lgmb_model.predict(X_test)
real_y_pred = lgmb_model.predict(test_X)

In [ ]:
train_y_pred.shape

In [ ]:
#log_loss
log_loss(y_test,train_y_pred,labels= list(set(y_test)))

In [ ]:
lgmb_model.feature_importance()

### feature_importance

In [ ]:
pi_df = pd.DataFrame(lgmb_model.feature_importance(), index= train_X.columns,columns= ['importance']).sort_values('importance',ascending = False)
pi_df.head(10)

### 예측결과에 대해
* 12번이 모두 6번으로 몰림
중간예측 : 0(STAR_WHITE_DWARF-2160),2(STAR_BROWN_DWARF-500),7(SERENDIPITY_DISTANT-4654),11(ROSAT_D-6580)
망한예측 : 12(STAR_PN-13),17(SERENDIPITY_MANUAL-61)

In [ ]:
cm_result = pd.DataFrame(confusion_matrix(y_test,np.argmax(train_y_pred,axis = 1)))

In [ ]:
number_columns

In [ ]:
train_df.type.value_counts()

In [ ]:
cm_result

In [ ]:
plt.figure(figsize =(15,15))
sns.heatmap(cm_result/np.sum(cm_result),annot = True, cmap ='RdYlGn',fmt='.1%')
plt.ylabel("real",fontsize=20)
plt.xlabel("predict",fontsize=20)
plt.show()

# REAL

In [ ]:
train_ds = lgb.Dataset(train_X, label = train_y)

In [ ]:
lgmb_model = lgb.train(params, train_ds,num_boost_round= 2000, verbose_eval=100)

In [ ]:
y_pred = lgmb_model.predict(test_X)

In [ ]:
y_pred

In [51]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
submission.to_csv('./submisson/submission.csv', index=True)